In [ ]:
import sys

import numpy as np
import pandas as pd

# ^^^ pyforest auto-imports - don't write above this line
try:
    import bib_lookup
except ModuleNotFoundError:
    sys.path.insert(0, "/home/wenhao/Jupyter/wenhao/workspace/bib_lookup/")
try:
    from torch_ecg.utils.misc import MovingAverage, list_sum
except ModuleNotFoundError:
    sys.path.insert(0, "/home/wenhao/Jupyter/wenhao/workspace/torch_ecg/")
    from torch_ecg.utils.misc import MovingAverage, list_sum

In [ ]:
from copy import deepcopy

from cfg import ModelCfg, ModelCfg_ns, TrainCfg, TrainCfg_ns
from dataset import CINC2021
from model import ECG_CRNN_CINC2021
from torch.nn.parallel import DataParallel as DP
from torch.nn.parallel import DistributedDataParallel as DDP
from tqdm.auto import tqdm
from trainer import CINC2021Trainer

In [ ]:
ECG_CRNN_CINC2021.__DEBUG__ = False
CINC2021Trainer.__DEBUG__ = False
CINC2021.__DEBUG__ = False

In [ ]:
TrainCfg_ns.db_dir = "/home/wenhao/Jupyter/wenhao/data/CinC2021/"
TrainCfg.db_dir = "/home/wenhao/Jupyter/wenhao/data/CinC2021/"

In [ ]:
ds_train = CINC2021(TrainCfg_ns, training=True, lazy=False)
ds_val = CINC2021(TrainCfg_ns, training=False, lazy=False)

## 12 lead, resnet_nature_comm_bottle_neck_se, 1-linear, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.cnn_name = "resnet_nature_comm_bottle_neck_se"
train_config.rnn_name = "none"
train_config.attn_name = "none"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name
model_config.clf = ED()
model_config.clf.out_channels = [
    # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

## collect results

In [ ]:
import matplotlib.patches as patches
import seaborn as sns
from matplotlib.pyplot import cm

sns.set()

plt.rcParams["xtick.labelsize"] = 28
plt.rcParams["ytick.labelsize"] = 28
plt.rcParams["axes.labelsize"] = 40
plt.rcParams["legend.fontsize"] = 24

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

markers = ["p", "v", "s", "d", "x", "*", "+", "$\heartsuit$"]
marker_size = 12

In [ ]:
from torch_ecg.utils.misc import MovingAverage, list_sum

In [ ]:
# ma = MovingAverage()
ma_ea = MovingAverage()

ma = lambda x: x

In [ ]:
df_ms = pd.read_csv("./results/TorchECG_11-19_22-31_ECG_CRNN_CINC2021_adamw_amsgrad_LR_0.0001_BS_64_multi_scopic.csv")
df_ms_lw = pd.read_csv(
    "./results/TorchECG_11-24_00-21_ECG_CRNN_CINC2021_adamw_amsgrad_LR_0.0001_BS_64_multi_scopic_leadwise.csv"
)

In [ ]:
df_ms_train = df_ms[df_ms.part == "train"].dropna(subset=["challenge_metric"])
df_ms_lw_train = df_ms_lw[df_ms_lw.part == "train"].dropna(subset=["challenge_metric"])
df_ms_val = df_ms[df_ms.part == "val"]
df_ms_lw_val = df_ms_lw[df_ms_lw.part == "val"]

In [ ]:
df_lr = df_ms[
    [
        "epoch",
        "loss",
        "lr",
        "part",
        "step",
    ]
]
df_lr.step = (df_lr.step / 20) / 53

In [ ]:
fig, ax = plt.subplots(figsize=(20, 12))

line_width = 4

lines = []

spacing = 2
loss_spacing = 16

lines.append(
    ax.plot(
        df_ms_train.epoch.values[::spacing],
        ma(df_ms_train.challenge_metric.values)[::spacing],
        marker=markers[0],
        markersize=marker_size,
        linewidth=line_width,
        color=colors[0],
        ls="dashed",
        label="train",
    )
)
lines.append(
    ax.plot(
        df_ms_lw_train.epoch.values[::spacing],
        ma(df_ms_lw_train.challenge_metric.values)[::spacing],
        marker=markers[1],
        markersize=marker_size,
        linewidth=line_width,
        color=colors[1],
        ls="dashed",
        label="lw-train",
    )
)
lines.append(
    ax.plot(
        df_ms_val.epoch.values[::spacing],
        ma(df_ms_val.challenge_metric.values)[::spacing],
        marker=markers[0],
        markersize=marker_size,
        linewidth=line_width,
        color=colors[0],
        ls=(0, (1, 1)),
        label="val",
    )
)
lines.append(
    ax.plot(
        df_ms_lw_val.epoch.values[::spacing],
        ma(df_ms_lw_val.challenge_metric.values)[::spacing],
        marker=markers[1],
        markersize=marker_size,
        linewidth=line_width,
        color=colors[1],
        ls=(0, (1, 1)),
        label="lw-val",
    )
)
ax.set_ylim(0.35, 1.05)
ax.set_xlabel("Epochs (n.u.)", fontsize=36)
ax.set_ylabel("Challenge score (n.u.)", fontsize=36)

ax2 = ax.twinx()
df_tmp = df_ms[(~df_ms.loss.isna())]
df_tmp.step = (df_tmp.step / 20) / 53
lines.append(
    ax2.plot(
        df_tmp.step.values[::loss_spacing],
        ma_ea(df_tmp.loss.values)[::loss_spacing],
        color=colors[0],
        ls="-",
        linewidth=line_width,
        label="train-loss",
    )
)
df_tmp = df_ms_lw[(~df_ms_lw.loss.isna())]
df_tmp.step = (df_tmp.step / 20) / 53
lines.append(
    ax2.plot(
        df_tmp.step.values[::loss_spacing],
        ma_ea(df_tmp.loss.values)[::loss_spacing],
        color=colors[1],
        ls="-",
        linewidth=line_width,
        label="lw-train-loss",
    )
)
ax2.set_ylabel(r"Loss (n.u.)")
ax2.set_ylim(-0.03, 0.39)
ax2.set_yticks(np.arange(0, 0.42, 0.06).tolist())

lns = list_sum(lines)
labs = [l.get_label() for l in lns]
ax.legend(
    lns,
    labs,
    loc="upper center",
    bbox_to_anchor=(0.5, -0.1),
    ncol=3,
    fancybox=False,
    shadow=False,
    fontsize=30,
)
lr_line = ax.plot(
    df_lr.step.values[::spacing],
    (df_lr.lr.values / df_lr.lr.max() / 1.8 + 0.4)[::spacing],
    linestyle=":",
    linewidth=6,
    color=colors[2],
)
ax.text(13, 0.97, "max lr = 0.02", fontsize=30)
ax.text(2, 0.41, f"start lr = {df_lr.lr.values[0]:.5f}", fontsize=30)
ax2.legend(
    lr_line,
    [
        "learning rate",
    ],
    loc="upper right",
    fontsize=30,
    bbox_to_anchor=(0.95, 1),
)

plt.savefig("./results/cinc2021_nn_compare.svg", dpi=1200, bbox_inches="tight", transparent=False)
plt.savefig("./results/cinc2021_nn_compare.pdf", dpi=1200, bbox_inches="tight", transparent=False);